In [1]:
import pandas as pd
import numpy as np

In [112]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
links=pd.read_csv('links.csv')
tags=pd.read_csv('tags.csv')

### Create movie profile by genres. each columns is 0/1 indicator for each genre

In [3]:
movies2=movies['genres'].apply(lambda x: x.split('|'))

In [4]:
##Create a list of unique Movie Genres
unique_genres=list()
for x in range(len(movies2)):
    for y in range(len(movies2[x])):
        if movies2[x][y] not in unique_genres:
            unique_genres.append(movies2[x][y])

In [5]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [6]:
movies3=pd.DataFrame(movies2)

In [7]:
movies3['genres'].head(2)

0    [Adventure, Animation, Children, Comedy, Fantasy]
1                       [Adventure, Children, Fantasy]
Name: genres, dtype: object

In [8]:
for z in unique_genres:
    movies3[z]=0

In [9]:
for x in range(len(movies)):
    for y in range(len(movies3.columns)):
        if str(movies3.columns[y]) in str(movies3.iloc[x,0]):
            movies3.iloc[x,y]=1

In [10]:
movies3.head(3)

,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[Comedy, Romance]",0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Use numpy to calculate the similarity matrix (m x m)
#### i) Normalize each row by norm (A/|A| etc)


In [11]:
movies3['Norm']=np.sqrt(np.count_nonzero(movies3.iloc[:,1:],axis=1))

In [12]:
movies3.head(3)

,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),Norm
0,"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.236068
1,"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.732051
2,"[Comedy, Romance]",0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1.414214


In [23]:
movies4=movies3.iloc[:,1:]

In [24]:
for i in movies4.columns:
    movies4[i]=movies4[i]/movies4['Norm']

In [26]:
movies4=movies4.iloc[:,:-1]

### Use numpy to calculate the similarity matrix (m x m)
#### ii) Obtain similarity matrix by M dot Mt

In [29]:
similarity_matrix=np.dot(movies4, movies4.T)

In [281]:
similarity_matrix2=pd.DataFrame(similarity_matrix,columns=movies['movieId']) #Remeber: index is equal to genres

In [282]:
similarity_matrix2=pd.merge(similarity_matrix2,movies,left_on=similarity_matrix2.index,right_on=movies.index,how='inner')

In [283]:
similarity_matrix2=similarity_matrix2.set_index(drop=True,keys='movieId')

In [284]:
similarity_matrix2.head(5)

,key_0,1,2,3,4,5,6,7,8,9,...,193571,193573,193579,193581,193583,193585,193587,193609,title,genres
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,...,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,...,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000,Jumanji (1995),Adventure|Children|Fantasy
3,2,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,...,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107,Grumpier Old Men (1995),Comedy|Romance
4,3,0.258199,0.000000,0.816497,1.000000,0.577350,0.0,0.816497,0.000000,0.0,...,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350,Waiting to Exhale (1995),Comedy|Drama|Romance
5,4,0.447214,0.000000,0.707107,0.577350,1.000000,0.0,0.707107,0.000000,0.0,...,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000,Father of the Bride Part II (1995),Comedy


In [285]:
similarity_matrix3=similarity_matrix2.iloc[:,1:-2]

In [286]:
similarity_matrix3.head(3)

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.0,0.316228,0.447214
2,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.0,0.000000,0.000000
3,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.0,0.000000,0.707107


In [287]:
pd.DataFrame(movies.movieId.unique())[0].drop_duplicates #Confirm no duplicate

<bound method Series.drop_duplicates of 0            1
1            2
2            3
3            4
4            5
         ...  
9737    193581
9738    193583
9739    193585
9740    193587
9741    193609
Name: 0, Length: 9742, dtype: int64>

### Write a function to get movie id and return the top K most similar movies with
#### 1) min score, max score, 2)min rating, 3)min total rating, 4)time range

In [306]:
def top_K_Similar():
    movie_select=input('Please enter a movieid: ')
    TopK=input('How many top movies do you want to know: ')
    r=int(movie_select)
    k=int(TopK)
    sort_c=pd.DataFrame(similarity_matrix3[r].sort_values(ascending=False))
    sort_c2=sort_c[:k]
    output1=pd.merge(sort_c2,movies,left_on=sort_c2.index,right_on='movieId',how='inner')
    output1=output1.iloc[:,1:-1]
    return output1

In [307]:
top_K_Similar()

Please enter a movieid:  1
How many top movies do you want to know:  10


,movieId,title
0,1,Toy Story (1995)
1,136016,The Good Dinosaur (2015)
2,3114,Toy Story 2 (1999)
3,91355,Asterix and the Vikings (Astérix et les Viking...
4,65577,"Tale of Despereaux, The (2008)"
5,45074,"Wild, The (2006)"
6,103755,Turbo (2013)
7,4886,"Monsters, Inc. (2001)"
8,166461,Moana (2016)
9,3754,"Adventures of Rocky and Bullwinkle, The (2000)"


In [275]:
##Checking
similarity_matrix3.iloc[0,:].sort_values(ascending=False)

1         1.0
136016    1.0
3114      1.0
91355     1.0
65577     1.0
         ... 
174403    NaN
176601    NaN
181413    NaN
181719    NaN
182727    NaN
Name: 1, Length: 9742, dtype: float64

In [294]:
##Checking 2
similarity_matrix2[1].sort_values(ascending=False)

movieId
1         1.0
136016    1.0
3114      1.0
91355     1.0
65577     1.0
         ... 
174403    NaN
176601    NaN
181413    NaN
181719    NaN
182727    NaN
Name: 1, Length: 9742, dtype: float64